# Import

In [ ]:
import pandas as pd
import os
import key_param as key_param

# Setup

In [ ]:
os.environ["OPENAI_API_KEY"] = key_param.OPENAI_API_KEY

from openai import OpenAI
client = OpenAI()

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
   text = str(text)
   text = text.replace("\n", " ")
   
   return client.embeddings.create(input = [text], model=model).data[0].embedding

# Function

In [ ]:
def combined_Description(df):
    """
    เป็นการเลือก Column เพื่อนำไป Embedding
    """
    combined = []
    
    for _, row in df.iterrows():
        row_values = [str(value).strip() if not pd.isna(value) else "" for value in row]  # ใช้ทุก column
        combined.append(" ".join(row_values))
    
    df["combined"] = combined
    
    return df

In [ ]:
def embeddingColumn(df):
    """
    นำ Column ที่เลือกมา มา Embedding
    """
     
    df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
     
    return df

In [ ]:
def to_structure(df):
    """
    เปลี่่ยนตาราง CSV ให้เป็นรูปแบบ Column : Value ตาม pipeline
    """
    df_s_col_list = df.columns.tolist()
    
    sturcture_text_list = []
    for j in range(len(df)):
        sturcture_text = ""  
        df_s_row_list = df.iloc[j,:].tolist()
        for i in range(len(df_s_col_list)):
            sturcture_text += str(df_s_col_list[i]) + ' : ' + str(df_s_row_list[i]) + '\n'
            
            
        sturcture_text_list.append(sturcture_text)
        
    
    df['Content'] = sturcture_text_list
    
        
    return df    

In [ ]:
def to_vectore_store(df):
    """
    สร้าง Vector Store
    """
    
    df = combined_Description(df)
    df = embeddingColumn(df)
    df = to_structure(df)
    
    return df

# Make Vector Store

In [ ]:
df = pd.read_csv('./Main.csv', encoding='utf-8')
df

In [ ]:
vector_store = to_vectore_store(df)
vector_store

In [ ]:
vector_store.to_csv("CSV_VectorStore.csv", index=False)